In [164]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

# 1 Get all players

In [ ]:
base_url = 'https://www.elost.cz/st/zebricek/cr/stranka-'
players = []

for i in range(156):
    if i == 0: continue
    url = base_url + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    games = str(soup.find_all("table", class_="rank"))
    p = re.compile(r'''<a href=(["'])(.*?)\1''')
    r = p.findall(games)
    rl = [el[1] for el in r]
    players.append(rl)

In [111]:
flatten = [item for sublist in players for item in sublist] # flatten list of lists
players_only = list(filter(lambda url: url.find("oddil") <= 0, flatten)) # filter out team urls (could have been done in previous step)

In [112]:
# save players list to file just in case..
with open('data/1_players.txt', 'w') as f:
    for player in players_only:
        f.write("%s\n" % player)

In [107]:
# with open('data/1_players.txt', 'r') as f:
#     l = f.read().splitlines()

# 2 Download games for each player and create result dataframe with all games

In [216]:
result = pd.DataFrame()
wait = False

for i, player in enumerate(players_only):
    if len(result) > 1000000: result = pd.DataFrame() # new file for each 1M rows
    print(str(i), '|', player)
    try:
        year = int(player[-4:])
    except:
        year = int(player[-6:-2]) # some players have suffix behing ther ID
    url = f'https://www.elost.cz{player}?filtr_i_vyhry=ano&filtr_i_prohry=ano&filtr_od_sezony=2011&filtr_zapasy_search=podle+soupe%C5%99e+nebo+odd%C3%ADlu#hrac_zapasy'
    
    # wait in case response was too long
    if wait:
        print("waiting 8 seconds")
        time.sleep(8)
    
    # measure response
    start_player = time.time()
    page = requests.get(url)
    end_player = time.time()
    response = end_player - start_player
    print(response)
    if response > 5: 
        wait = True
    else:
        wait = False
    
    # get content
    soup = BeautifulSoup(page.content, 'html.parser')
    games = str(soup.find_all("table", class_="rank"))
    d = games.replace('<br/>', '|')
    df = pd.read_html(d)[1]

    # parse columns
    df[['Hráč', 'Oddíl', 'Družstvo']] = df['Hráč|Oddíl|Družstvo'].str.split('|', expand=True)
    df[['Soupeř', 'Oddíl soupeř', 'Družstvo soupeř']] = df['Soupeř|Oddíl|Družstvo'].str.split('|', expand=True)
    df[['Výsledek', 'Datum zápasu', 'Soutěž']] = df['Výsledek|Datum zápasu|Soutěž'].str.split('|', expand=True)
    df[['Elo hráče', 'Elo (min)', 'Elo (max)']] = df['Elo hráče  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
    df[['Elo soupeře', 'Elo (min) soupeře', 'Elo (max) soupeře']] = df['Elo soupeře  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
    df['Rok narození'] = year

    # get additional information
    main_section = soup.find("div", {"id": "main_section"}).text
    try:
        df['Max elo'] = int(re.compile("Nejvyšší dosažené Elo: [0-9]{3,4}").findall(main_section)[0][-4:])
    except:
        df['Max elo'] = None
    try:
        df['Elo nejlepšího poraženého soupeře'] = int(re.compile("Nejlépe hodnocený hráč, kterého .* porazil\(a\): .*, Elo: ([0-9]{3,4})").findall(main_section)[0])
    except:
        df['Elo nejlepšího poraženého soupeře'] = None

    df['ID Hráč'] = player
    df['ID Soupeř'] = re.compile('st/hrac/(.*)"').findall(d)
    df['Rok narození soupeř'] = df['ID Soupeř'].apply(lambda x: int(x[-4:]) if (x[-4:].find("-") <= 0) else int(x[-6:-2]))

    # add player games & information to previous collection
    player_games = df.iloc[:, 5:]
    result = result.append(player_games)

    # save batch for each 200 players
    if i % 200 == 0:
        suffix = str(dt.now().time()).replace(":", "_")
        result.to_csv(f'v:\\My Drive\\_VSE\\_MGR\\3sem\\3_4IT574_Prakticke aplikace strojoveho uceni\\-VSE-Applied_ML\\data\\games_checkpoint\\games_all_{suffix}.csv')

0 | /st/hrac/lipovsky_jaroslav_1964
0.6148571968078613
1 | /st/hrac/milan_safranek_1965
0.2846057415008545
2 | /st/hrac/vavrinec_jiri_1982
0.5901641845703125
3 | /st/hrac/rabochova_hana_1975
0.42504119873046875
4 | /st/hrac/michal_skoda_1972
0.34319424629211426
5 | /st/hrac/jiri_pessr_1949
0.4211854934692383
6 | /st/hrac/sevl_vladimir_1971
0.436992883682251
7 | /st/hrac/ondrej_pospisil_2009
0.34908270835876465
8 | /st/hrac/stepan_sedlacek_2008
0.2508277893066406
9 | /st/hrac/jan_vedral_1953
0.38019442558288574
10 | /st/hrac/yaroslavtsev_vitaly_1959
0.710296630859375
11 | /st/hrac/josef_studeny_2009
0.34704017639160156
12 | /st/hrac/miklik_pavel_1974
0.7004950046539307
13 | /st/hrac/jakub_pelant_1980
0.3259556293487549
14 | /st/hrac/novotny_dalimil_1995
0.2728278636932373
15 | /st/hrac/bacevsky_josef_1948
0.229996919631958
16 | /st/hrac/pavel_czina_2005
0.3675878047943115
17 | /st/hrac/volek_jan_1982
0.6850051879882812
18 | /st/hrac/tylsar_jan_2003
0.4893670082092285
19 | /st/hrac/rosko